In [1]:
DATA_PATH = '../../data/uniprot-reviewed%3Ayes.tab.gz'
MAX_SEQUENCE_LENGTH = 500

In [2]:
AMINO_ACIDS_SET = {'G', 'A', 'V', 'L', 'I', 'P', 'F', 'Y', 'W', 'S', 'T', 'C', 'M', 'N', 'Q', 'K', 'R', 'H', 'D', 'E'}

# Data Loading

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(DATA_PATH, sep='\t', header=(0), skipinitialspace=True)

In [5]:
df.head()

,Entry,Entry name,Gene names,Length,Sequence,EC number
0,Q754G5,ARP4_ASHGO,ARP4 AFR105C,479,MSNSALQVYGGDEITAVVIDPGSFTTNIGYSGTDCPQAILPSCYGK...,NaN
1,P19108,ARRB_DROMI,Arr2 ArrB,401,MVVSVKVFKKATPNGKVTFYLGRRDFIDHLDYCDPVDGVIVVEPEY...,NaN
2,P36575,ARRC_HUMAN,ARR3 ARRX CAR,388,MSKVFKKTSSNGKLSIYLGKRDFVDHVDTVEPIDGVVLVDPEYLKC...,NaN
3,Q96B67,ARRD3_HUMAN,ARRDC3 KIAA1376,414,MVLGKVKSLTISFDCLNDSNVPVYSSGDTVSGRVNLEVTGEIRVKS...,NaN
4,P63621,ARSC_NEIMA,arsC NMA0252,117,MPEIKIFHNPRCSKSRAALSLLEERGIAAEVVKYLDTPPDLSELKD...,1.20.4.1


In [6]:
print('Rows count: {}'.format(df.shape[0]))

Rows count: 556568


# Data PreProcessing

## Handling empty values

In [7]:
print('Empty values in columns')
df.isnull().sum()

Empty values in columns


Entry              0
Entry name         0
Gene names     22953
Length             0
Sequence           0
EC number     293609
dtype: int64

In [8]:
print('Removing rows that does not have EC number')
df_nonan = df.dropna(axis=0, how='any', subset=['EC number'])
dropped_rows_count = df.shape[0] - df_nonan.shape[0]
print('Rows count before: {}'.format(df.shape[0]))
print('Rows count after: {}'.format(df_nonan.shape[0]))
print('Rows removed: {} ({:0.2f}%)'.format(dropped_rows_count, dropped_rows_count/df.shape[0]*100))

Removing rows that does not have EC number
Rows count before: 556568
Rows count after: 262959
Rows removed: 293609 (52.75%)


## Handling Non-Standard amino acids

In [9]:
df_noamino = df_nonan[df_nonan["Sequence"].apply(lambda x: set(x).issubset(AMINO_ACIDS_SET))]

In [10]:
dropped_rows_count = df_nonan.shape[0] - df_noamino.shape[0]
print('Rows count before: {}'.format(df_nonan.shape[0]))
print('Rows count after: {}'.format(df_noamino.shape[0]))
print('Rows removed: {} ({:0.2f}%)'.format(dropped_rows_count, dropped_rows_count/df_nonan.shape[0]*100))

Rows count before: 262959
Rows count after: 261991
Rows removed: 968 (0.37%)


## Dropping Too Long Sequences

In [11]:
df_maxlength = df_noamino[df_noamino["Length"] <= MAX_SEQUENCE_LENGTH]

In [12]:
dropped_rows_count = df_noamino.shape[0] - df_maxlength.shape[0]
print('Rows count before: {}'.format(df_noamino.shape[0]))
print('Rows count after: {}'.format(df_maxlength.shape[0]))
print('Rows removed: {} ({:0.2f}%)'.format(dropped_rows_count, dropped_rows_count/df_noamino.shape[0]*100))

Rows count before: 261991
Rows count after: 205843
Rows removed: 56148 (21.43%)


In [13]:
df_processed = df_maxlength

## Adding Multi-Level EC Numbers

In [14]:
def generateLabel(item, index):
    return ".".join(item.split(".")[0:index])       

In [15]:
for i in range(1,5):
    df_maxlength["Level_"+str(i)] = [ generateLabel(item[0],i) for item in df_maxlength["EC number"].str.split(";").values ]

/home/donatasrep/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
df_maxlength.head()

,Entry,Entry name,Gene names,Length,Sequence,EC number,Level_1,Level_2,Level_3,Level_4
4,P63621,ARSC_NEIMA,arsC NMA0252,117,MPEIKIFHNPRCSKSRAALSLLEERGIAAEVVKYLDTPPDLSELKD...,1.20.4.1,1,1.20,1.20.4,1.20.4.1
5,Q8ENQ5,ARSC_OCEIH,arsC OB2423,139,MSKKIIYFLCTGNSCRSQMAEGWGKKILGEEWDVYSAGIEAHGLNP...,1.20.4.4,1,1.20,1.20.4,1.20.4.4
6,Q2FFW6,ARSC_STAA3,arsC SAUSA300_1719,131,MTKKTIYFICTGNSCRSQMAEGWAKQILADDWNVYSAGIETHGVNP...,1.20.4.4,1,1.20,1.20.4,1.20.4.4
7,Q2YTL3,ARSC_STAAB,arsC SAB1632,131,MTKKTIYFICTGNSCRSQMAEGWAKQILAEDWNVYSAGIETHGVNP...,1.20.4.4,1,1.20,1.20.4,1.20.4.4
8,Q5HF01,ARSC_STAAC,arsC SACOL1824,131,MTKKTIYFICTGNSCRSQMAEGWAKQILADDWNVYSAGIETHGVNP...,1.20.4.4,1,1.20,1.20.4,1.20.4.4


# First level analysis

In [17]:
df_level1 = df_maxlength[['Level_1','Length']]

df_level1.groupby('Level_1').describe()

Length                                                          
           count        mean         std   min    25%    50%    75%    max
Level_1                                                                   
1        32142.0  307.944621  105.879715   4.0  228.0  324.0  380.0  500.0
2        76011.0  306.805699   93.772955   5.0  237.0  311.0  375.0  500.0
3        47778.0  277.298757  109.514130   6.0  194.0  266.0  354.0  500.0
4        20313.0  293.681534  109.662534   9.0  200.0  286.0  382.0  500.0
5        12144.0  310.981308  101.114842   9.0  239.0  287.0  419.0  500.0
6        17455.0  376.495560   94.701974  10.0  313.0  404.0  453.0  500.0

# Second Level Analysis

In [18]:
df_level2 = df_maxlength[['Level_2','Length']]

In [19]:
df_level2_group = df_level2.groupby('Level_2').filter(lambda x: len(x) > 10000)

In [20]:
df_level2_group.groupby('Level_2').describe()

Length                                                          
           count        mean         std   min    25%    50%    75%    max
Level_2                                                                   
2.1      16370.0  304.575443   78.600895   9.0  246.0  303.0  364.0  500.0
2.7      25785.0  289.602870  103.000477   9.0  212.0  289.0  367.0  500.0
3.1      16181.0  240.133304   98.545559   6.0  157.0  230.0  307.0  500.0
3.6      11339.0  303.761178  112.112808  14.0  207.0  285.0  390.0  500.0
6.3      10030.0  358.018744  106.683177  15.0  282.0  373.0  447.0  500.0

# Third level analysis

In [21]:
df_level3 = df_maxlength[['Level_3','Length']]
df_level3_group = df_level3.groupby('Level_3').filter(lambda x: len(x) > 7000)
df_level3_group.groupby('Level_3').describe()

Length                                                          
           count        mean         std   min    25%    50%    75%    max
Level_3                                                                   
2.1.1    12267.0  293.226135   80.627205   9.0  238.0  279.0  353.0  500.0
2.3.1     7242.0  319.683927   81.654030  10.0  262.0  335.0  380.0  500.0
2.5.1     7488.0  315.203526   84.263921  15.0  277.0  311.0  392.0  500.0
2.7.1     7093.0  303.488228   87.043300  14.0  256.0  298.0  337.0  500.0
2.7.7     7818.0  279.584165  119.015882  10.0  189.0  258.0  380.0  500.0

# Fourth level analysis

In [51]:
df_level4 = df_maxlength[['Level_4','Length']]
df_level4_group = df_level4.groupby('Level_4').filter(lambda x: len(x) > 100)
df_level4_group.groupby('Level_4').agg({"Length": ["mean", "count", "std", "max"]}).sort_values(by=[("Length","mean")])
#3.6.1.7

Length                      
                 mean count        std  max
Level_4                                    
1.97.1.12   80.652439   164   5.708238   93
3.6.1.7     93.623616   271   5.626527  120
4.2.1.96   105.253906   256  10.899990  192
3.6.1.31   105.712766   282  14.432039  252
3.5.1.5    109.148014   554  27.659577  308
3.1.1.4    117.848156   461  43.273482  239
2.4.2.2    122.970000   200  86.561037  433
2.7.8.7    129.514530   585  20.034650  344
3.1.26.5   130.956585   691  35.640327  366
5.4.99.62  135.072816   206   4.383955  154
3.1.27.5   140.336634   101  17.575140  161
4.2.3.3    140.817734   203  11.576319  158
3.5.4.19   142.413598   353  36.735990  417
1.8.4.12   144.669811   212  19.079639  291
2.7.4.6    144.916329   741  22.602165  395
4.1.1.11   145.358916   443  64.120008  398
2.8.1.1    149.845638   149  80.434439  435
4.2.1.59   155.438735   759  11.640101  184
2.1.1.177  155.686084   618   5.915820  174
5.3.1.26   157.071429   112  14.803040  173
2.5.1.78   157.253247   616   8.608985  227
4.2.1.104  157.338710   124   8.789209  205
3.6.1.23   161.368421   494  43.712386  436
4.6.1.12   161.651270   433   9.255672  240
4.6.1.17   162.539726   365  18.679385  347
3.1.1.96   162.810084   595  50.490649  493
2.7.7.3    163.175516   678  11.822425  328
4.4.1.21   163.288194   288   6.608800  173
3.4.23.36  166.610451   421  15.234212  304
3.2.1.17   166.638298   188  63.268776  480
...               ...   ...        ...  ...
2.1.1.107  447.774194   155  62.426221  496
2.1.1.74   448.015625   256  15.880675  482
2.8.4.4    448.712159   403  13.976047  495
3.2.1.18   449.041916   167  59.273651  484
5.4.2.10   449.249635   685   7.084114  490
6.3.2.9    452.217778   450  16.856836  500
6.1.1.22   453.104478   268  26.626215  492
2.7.7.23   454.797342   602  26.918045  500
3.3.1.1    458.554656   247  50.817104  496
2.8.4.3    459.174905   526  16.421583  499
2.9.1.1    460.291339   127  13.305990  475
1.4.4.2    462.136364   264  19.275183  500
4.3.2.1    462.654098   610  12.007798  499
6.1.1.15   463.537500   240  21.832537  500
3.6.3.14   464.497012  1004  60.241927  500
6.1.1.16   466.688406   690  18.346369  500
6.3.2.8    468.015221   657  17.782641  498
5.3.1.12   469.030534   131   4.531834  487
6.1.1.17   471.072423   718  25.387547  500
2.7.1.167  476.270000   200  13.401309  496
2.4.1.21   479.752475   303   8.559576  500
6.3.5.7    484.338182   550  12.182066  500
6.3.2.13   487.743590   117  11.327533  500
1.2.1.71   488.811024   127   3.795846  498
1.2.1.8    489.059829   117   2.506167  500
1.1.5.4    489.375000   104  38.814267  500
3.4.11.1   489.617647   238  43.202604  500
4.1.1.98   491.632000   125   3.495472  499
6.1.1.6    493.465116   172  37.236622  500
2.7.1.30   495.157025   242  31.153084  500

[429 rows x 4 columns]

# EC Number file processing

In [3]:
columns = ['Level_1', 'Level_2', 'Level_3', 'Level_4', 'Name', 'ToDelete']
ecs = pd.read_csv('../../data/ECNumbers.csv', sep='.', header=None, names = columns, dtype = str)

In [4]:
ecs = ecs.drop(['ToDelete'], axis=1)

In [22]:
ecs['Level_1'] = ecs['Level_1'].str.strip()
ecs['Level_2'] = ecs['Level_2'].str.strip()
ecs['Level_3'] = ecs['Level_3'].str.strip()
ecs['Level_4'] = ecs['Level_4'].str.strip()

In [23]:
ecs['ECNumber'] = ecs['Level_1'] + '.' + ecs['Level_2'] + '.' + ecs['Level_3']

In [25]:
ecs = ecs[['ECNumber', 'Level_1', 'Level_2', 'Level_3', 'Level_4', 'Name']]

In [26]:
ecs

,ECNumber,Level_1,Level_2,Level_3,Level_4,Name
0,1.-.-,1,-,-,-,Oxidoreductases
1,1.1.-,1,1,-,-,Acting on the CH-OH group of donors
2,1.1.1,1,1,1,-,With NAD(+) or NADP(+) as acceptor
3,1.1.2,1,1,2,-,With a cytochrome as acceptor
4,1.1.3,1,1,3,-,With oxygen as acceptor
5,1.1.4,1,1,4,-,With a disulfide as acceptor
6,1.1.5,1,1,5,-,With a quinone or similar compound as acceptor
7,1.1.9,1,1,9,-,With a copper protein as acceptor
8,1.1.98,1,1,98,-,"With other, known, acceptors"
9,1.1.99,1,1,99,-,With other acceptors


In [32]:
ecs.to_csv('ECNumbers.csv', sep='\t', header=True, index=False)